In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [17]:
numbers_df = pd.read_excel('Звонки_заявки.xlsx', dtype=str)

phones = numbers_df['Номер телефона'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
    
    
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y
            
            
numbers_df['Чистые_телефоны'] = [''.join(i) for i in phones2]
numbers_df.to_excel('Обработано.xlsx', index=False)

## Не забудь удалить дубликаты, даты проверь там